# Ejercicio de parcial
El GCPD recolecta la información de casos policiales que acontecen en Ciudad Gótica. Esta información se encuentra guardada en un archivo con el siguiente formato:

*(fecha, id_caso, descripción, estado_caso, categoria, latitud, longitud)*

Los posibels estados que puede tener un caso son 1: caso abierto, 2: caso resuelto, 3: cerrado sin resolucion.

Las fechas se encuentran en el formato YYYY-MM-DD.

Por otro lado el comisionado Gordon guarda un registro detallado sobre en cuales casos fue activada la batiseñal para pedir ayuda del vigilante, Batman. Esta información se encuentra en un archivo con el siguiente formato *(id_caso, respuesta)*, siendo campo respuesta si la señal tuvo una respuesta positiva (1) o negativa (0) de parte de él.

El sector encargado de las estadísticas oficiales del GCPD quiere analizar las siguientes situaciones:

a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.

b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

Resolver ambas consultas utilizando Pandas.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [47]:
#Armo el primer dataframe (fecha, id_caso, descripción, estado_caso, categoria, latitud, longitud)

valores = {
    'fecha':['2020-03-01','2020-09-03','2020-09-07','2020-10-10','2020-11-05'],
    'id_caso':[0,1,2,3,4],
    'descripción':['desc0','desc1','desc2','desc3','desc4'],
    'estado_caso':[3,1,3,2,2],
    'categoria':['robo de vehiculos','robo','hurto','hurto','asesinato'],
    'latitud':[2,6,3,4,2],
    'longitud':[2,7,9,2,1]
}

casos_policiales_df = pd.DataFrame(valores,columns=valores.keys())
casos_policiales_df.head()


,fecha,id_caso,descripción,estado_caso,categoria,latitud,longitud
0,2020-03-01,0,desc0,3,robo de vehiculos,2,2
1,2020-09-03,1,desc1,1,robo,6,7
2,2020-09-07,2,desc2,3,hurto,3,9
3,2020-10-10,3,desc3,2,hurto,4,2
4,2020-11-05,4,desc4,2,asesinato,2,1


In [48]:
#Armo el segundo dataframe  (id_caso, respuesta)

valores = {
    'id_caso':[0,1,2,3,4],
    'respuesta':[0,0,1,0,1]
}

respuesta_batman_df = pd.DataFrame(valores,columns=valores.keys())
respuesta_batman_df.head()

,id_caso,respuesta
0,0,0
1,1,0
2,2,1
3,3,0
4,4,1


## a) Las categorias que hayan incrementado su tasa de resolución al menos un 10% en el último trimestre, con respecto al trimestre anterior.

In [50]:
casos_policiales_df['fecha'] = pd.to_datetime(casos_policiales_df['fecha'],format='%Y-%m-%d')

In [51]:
#Consigo los trimestres
casos_policiales_df['trimestre'] = (casos_policiales_df['fecha'].dt.month / 3)
casos_policiales_df['trimestre'] = casos_policiales_df['trimestre'].apply(np.ceil) # Redondeo para arriba el valor
casos_policiales_df['trimestre'] = casos_policiales_df['trimestre'].astype(np.int)
casos_policiales_df.head()

,fecha,id_caso,descripción,estado_caso,categoria,latitud,longitud,trimestre
0,2020-03-01,0,desc0,3,robo de vehiculos,2,2,1
1,2020-09-03,1,desc1,1,robo,6,7,3
2,2020-09-07,2,desc2,3,hurto,3,9,3
3,2020-10-10,3,desc3,2,hurto,4,2,4
4,2020-11-05,4,desc4,2,asesinato,2,1,4


In [56]:
casos_ultimos_trimestres=casos_policiales_df[(casos_policiales_df['trimestre']==3) | (casos_policiales_df['trimestre']==4)]
#Me quede con los dos ultimos trimestres


In [57]:
aumento_10_resolucion = lambda estado_caso: (estado_caso == 2).mean() # Me da el promedio de la cantidad de casos resueltos

In [59]:
resolucion = casos_ultimos_trimestres.pivot_table(columns='trimestre',index='categoria',aggfunc=aumento_10_resolucion,values='estado_caso').fillna(0)
resolucion

trimestre,3,4
categoria,,
asesinato,0.0,1.0
hurto,0.0,1.0
robo,0.0,0.0


In [62]:
aumento_resolucion = resolucion.loc[resolucion[4]*0.1>resolucion[3]]
aumento_resolucion

trimestre,3,4
categoria,,
asesinato,0.0,1.0
hurto,0.0,1.0


In [66]:
aumento_resolucion.index.values.tolist() # Muestro las categorias

['asesinato', 'hurto']

## b) Tasa de participación de Batman por categoria, para los delitos contra la propiedad (que enmarcan las categorías incendio intencional, robo, hurto y robo de vehiculos).

In [67]:
conjunto_df = casos_policiales_df.merge(respuesta_batman_df,on='id_caso')
conjunto_df

,fecha,id_caso,descripción,estado_caso,categoria,latitud,longitud,trimestre,respuesta
0,2020-03-01,0,desc0,3,robo de vehiculos,2,2,1,0
1,2020-09-03,1,desc1,1,robo,6,7,3,0
2,2020-09-07,2,desc2,3,hurto,3,9,3,1
3,2020-10-10,3,desc3,2,hurto,4,2,4,0
4,2020-11-05,4,desc4,2,asesinato,2,1,4,1


In [69]:
#Me quedo con los delitos contra la propiedad
delitos_propiedad = conjunto_df['categoria'].str.contains('incendio intencional|robo|hurto|robo de vehiculos')
conjunto_delitos_propiedad = conjunto_df.loc[delitos_propiedad]
conjunto_delitos_propiedad

,fecha,id_caso,descripción,estado_caso,categoria,latitud,longitud,trimestre,respuesta
0,2020-03-01,0,desc0,3,robo de vehiculos,2,2,1,0
1,2020-09-03,1,desc1,1,robo,6,7,3,0
2,2020-09-07,2,desc2,3,hurto,3,9,3,1
3,2020-10-10,3,desc3,2,hurto,4,2,4,0


In [71]:
respuesta_por_categoria = conjunto_delitos_propiedad.groupby(by='categoria').agg({'respuesta':'mean'})
respuesta_por_categoria.rename(columns={'respuesta':'Tasa de participacion'})

,Tasa de participacion
categoria,
hurto,0.5
robo,0.0
robo de vehiculos,0.0
